In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")

In [3]:
from deepsvg.svglib.geom import Point
from deepsvg.svglib.svg import SVG
from deepsvg.svglib.svg_path import SVGPath
from deepsvg.svglib.utils import to_gif

from deepsvg.difflib.tensor import SVGTensor
from deepsvg.difflib.utils import *
from deepsvg.difflib.loss import *

In [4]:
import torch.optim as optim
import IPython.display as ipd
from moviepy.editor import ImageClip, concatenate_videoclips, ipython_display

In [5]:
import pickle
import csv
import pandas as pd 

# Save svg tensor in a .pkl file

In [6]:
def save_svg_tensor(file_path_name, file_obj):
    # save svg tensor data into .pkl file 
    with open(file_path_name, 'wb') as handle:
        pickle.dump(file_obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_svg_tensor(file_path_name):
    # load svg tensor data object according to the data path  
    with open(file_path_name, 'rb') as handle:
        svg_tensor_obj = pickle.load(handle)
        return svg_tensor_obj

In [7]:
svg_folder_path = 'dataset/svgs_simplified/'
svg_tensor_folder_path = 'dataset/svgs_tensor/'

In [8]:
svg_file_name_list = os.listdir(svg_folder_path)

In [9]:
# load svg_meta.csv
nonempty_svg_data_list = []
with open('dataset/svgs_meta.csv','rt')as f:
    svg_meta_data = csv.reader(f)
    header = next(svg_meta_data)
    for row in svg_meta_data:
        svg_id = str(row[0])
        element_num = int(row[2])
        svg_file_name = svg_id + '.svg'
        nonempty_svg_data_list.append([svg_file_name, element_num])

In [10]:
for svg_file_attr_array in nonempty_svg_data_list:
    svg_file_name = svg_file_attr_array[0]
    element_num = svg_file_attr_array[1]
    # read the original svg object from simplified folder
    svg_file_path_name = os.path.join(svg_folder_path, svg_file_name)
    svg_obj = SVG.load_svg(svg_file_path_name)
    # transform svg object to tensor obect
    svg_tensor_content_obj = svg_obj.to_tensor()
    svg_tensor_color_list = [0] * element_num
    svg_tensor_obj = {"tensors": svg_tensor_content_obj, "fillings": svg_tensor_color_list}
    svg_tensor_file_name = svg_file_name.replace('.svg', '.pkl')
    svg_tensor_file_path_name = os.path.join(svg_tensor_folder_path, svg_tensor_file_name)
    # save tensor object to the file    
    save_svg_tensor(svg_tensor_file_path_name, svg_tensor_obj)

In [11]:
print(len(nonempty_svg_data_list))

968


Test the content of svg tensort object, it should contain two attributes, tensorts and fillings

In [13]:
svg_tensor_path = "dataset/svgs_tensor/10.pkl"
svg_tensor_obj = load_svg_tensor(svg_tensor_path)
print("svg_tensor_obj", svg_tensor_obj)

svg_tensor_obj {'tensors': tensor([[  0.0000,  -1.0000,  -1.0000,  ...,  -1.0000, 219.0279, 218.7935],
        [  3.0000,  48.5100,  48.5100,  ...,  -1.0000, 315.9435, 214.2935],
        [  0.0000,  -1.0000,  -1.0000,  ...,  -1.0000, 219.0567, 219.1853],
        ...,
        [  3.0000,   0.9235,   0.9235,  ...,  -1.0000, 229.9581, 122.5406],
        [  0.0000,  -1.0000,  -1.0000,  ...,  -1.0000, 224.4227, 122.4029],
        [  3.0000,   2.7701,   2.7701,  ...,  -1.0000, 229.9630, 122.4029]]), 'fillings': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [14]:
icon_tensor_path = "dataset/icons_tensor/0.pkl"
icon_tensor_obj = load_svg_tensor(icon_tensor_path)
print("icon_tensor_obj", icon_tensor_obj)

icon_tensor_obj {'tensors': [[tensor([[  0.,  -1.,  -1.,  -1.,  -1.,  -1.,  23.,  64.,  -1.,  -1.,  -1.,  -1.,
          81., 119.],
        [  1.,  -1.,  -1.,  -1.,  -1.,  -1.,  81., 119.,  -1.,  -1.,  -1.,  -1.,
          81., 119.],
        [  2.,  -1.,  -1.,  -1.,  -1.,  -1.,  81., 119.,  93., 122.,  96., 125.,
         100., 138.],
        [  1.,  -1.,  -1.,  -1.,  -1.,  -1., 100., 138.,  -1.,  -1.,  -1.,  -1.,
          81., 157.],
        [  2.,  -1.,  -1.,  -1.,  -1.,  -1.,  81., 157.,  77., 156.,  74., 156.,
          71., 154.],
        [  2.,  -1.,  -1.,  -1.,  -1.,  -1.,  71., 154.,  59., 147.,  59., 128.,
          71., 121.],
        [  2.,  -1.,  -1.,  -1.,  -1.,  -1.,  71., 121.,  74., 120.,  77., 120.,
          81., 119.],
        [  1.,  -1.,  -1.,  -1.,  -1.,  -1.,  81., 119.,  -1.,  -1.,  -1.,  -1.,
          81., 119.]]), tensor([[  0.,  -1.,  -1.,  -1.,  -1.,  -1.,  23.,  64.,  -1.,  -1.,  -1.,  -1.,
         129., 119.],
        [  1.,  -1.,  -1.,  -1.,  -1.,  -

# svgs_tensor process

In [12]:
def save_csv_file(csv_file_path_name, row_list):
    with open(csv_file_path_name,'rt') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in row_list:
            writer.writerow(row)

In [13]:
svg_tensor_processed = []
svg_tensor_folder_path_processed = 'dataset/svgs_tensor_processed/'
svgs_meta_processed_path_name = 'dataset/svgs_meta_processed.csv'
with open('dataset/svgs_meta.csv','rt')as f:
    svg_meta_data = csv.reader(f)
    header = next(svg_meta_data)
    svg_tensor_processed.append(header)
    index = 0
    for row in svg_meta_data:
        svg_tensor_file_name = row[0] + '.pkl'
        svg_tensor_file_name_processed = str(index) + '.pkl'
        svg_tensor_file_path_name = os.path.join(svg_tensor_folder_path, svg_tensor_file_name)
        svg_tensor_file_path_name_processed = os.path.join(svg_tensor_folder_path_processed, svg_tensor_file_name_processed)
        # load svg tensor object
        svg_tensor_obj = load_svg_tensor(svg_tensor_file_path_name)
        # save svg tensor object
        save_svg_tensor(svg_tensor_file_path_name_processed, svg_tensor_obj)
        # reorder the first index 
        row[0] = str(index)
        svg_tensor_processed.append(row)
        index = index + 1

In [15]:
# save the processed svg tensor
pd.DataFrame(svg_tensor_processed).to_csv(svgs_meta_processed_path_name, header=None, index=None)